In [1]:
import torch
import json

In [2]:
import os
print(os.getcwd())
# Please change the root to an absolute or relative path to DomiKnowS root.
# In case relative path is used, consider the printed `CWD` as current working directory.
root = '/home/hfaghihi/Framework/DomiKnowS'
import sys
sys.path.append(root)
from typing import Any, Dict

/home/hfaghihi/Framework/DomiKnowS/examples/Propara


In [3]:
with open("test_data.json", "r") as file:
    data = json.load(file)
data[0]

{'id': '37',
 'entities': ['plant; animal', 'soft tissues', 'bones', 'mineral', 'fossils'],
 'steps': ['A plant of animal dies in a watery environment.',
  'Is buried in mud and silt.',
  'Soft tissues quickly decompose leaving behind hard bones or shells.',
  'Over time sediment builds over the top and hardens into rock.',
  'As the bone decays mineral seeps in replacing it.',
  'Fossils are formed.'],
 'entity_step': [[[0.9683084487915039,
    0.029291482642292976,
    0.0024000774137675762],
   [0.012266993522644043, 0.002779645612463355, 0.9849532842636108],
   [0.006055360194295645, 0.0023835168685764074, 0.9915611147880554],
   [0.0024350169114768505, 0.0022026486694812775, 0.9953623414039612],
   [0.0015909943031147122, 0.0026168148033320904, 0.9957921504974365],
   [0.0015094410628080368, 0.002839647000655532, 0.9956509470939636],
   [0.0016313738888129592, 0.0032306257635354996, 0.9951379895210266]],
  [[0.9757811427116394, 0.01820589229464531, 0.006012959871441126],
   [0.064

In [4]:
from regr.data.reader import RegrReader

In [5]:
class ProparaReader(RegrReader):
    def getprocedureIDval(self, item):
        return item['id']
    def getentitiesval(self, item):
        return item['entities']
    def getstepsval(self, item):
        num_steps = len(item['steps'])
        rel = torch.ones(num_steps,1)
        return rel, item['steps']
    def getentity_stepval(self, item):
        steps = len(item['steps'])
        entities = len(item['entities'])
        entity_step = []
        arg1s = []
        arg2s = []
        for entity,eraw in enumerate(item['entities']):
            arg1 = torch.zeros(entities)
            arg1[entity] = 1
            for step,sraw in enumerate(item['steps']):
                arg2 = torch.zeros(steps)
                arg2[step] = 1
                arg1s.append(arg1)
                arg2s.append(arg2)
                entity_step.append((eraw, sraw))
        return torch.stack(arg1s), torch.stack(arg2s), entity_step
    
    def getnon_existenceval(self, item):
        values = []
        for entity, eraw in enumerate(item['entities']):
            for step, sraw in enumerate(item['steps']):
                values.append([1 - item['entity_step'][entity][step][2], item['entity_step'][entity][step][2]])
        return torch.tensor(values)
            
    def getknownval(self, item):
        values = []
        for entity, eraw in enumerate(item['entities']):
            for step, sraw in enumerate(item['steps']):
                values.append([1 - item['entity_step'][entity][step][0], item['entity_step'][entity][step][0]])
        return torch.tensor(values)
    
    def getunkownval(self, item):
        values = []
        for entity, eraw in enumerate(item['entities']):
            for step, sraw in enumerate(item['steps']):
                values.append([1 - item['entity_step'][entity][step][1], item['entity_step'][entity][step][1]])
        return torch.tensor(values)
    
    def getactionval(self, item):
        action1s = torch.diag(torch.ones( len(item['entities']) * len(item['steps']) ) )[:-1]
        action2s = torch.diag(torch.ones( len(item['entities']) * len(item['steps']) ) )[1:]
        return action1s, action2s
    
    def getcreateval(self, item):
        actions = []
        for eid, entity in enumerate(item['entities']):
            for sid, step in enumerate(item['steps']):
                o = 0
                c = 0
                d = 0
                if sid == 0:
                    prev_state = item['entity_step'][eid][sid]
                    continue
                else:
                    o += (prev_state[0] * item['entity_step'][eid][sid][0])
                    o += (prev_state[0] * item['entity_step'][eid][sid][1])
                    o += (prev_state[1] * item['entity_step'][eid][sid][0])
                    o += (prev_state[1] * item['entity_step'][eid][sid][1])
                    o += (prev_state[2] * item['entity_step'][eid][sid][2])
                    d += (prev_state[0] * item['entity_step'][eid][sid][2])
                    d += (prev_state[1] * item['entity_step'][eid][sid][2])
                    c += (prev_state[2] * item['entity_step'][eid][sid][1])
                    c += (prev_state[2] * item['entity_step'][eid][sid][0])
                    actions.append([1-c, c])
                    prev_state = item['entity_step'][eid][sid]
        return actions
                    
    def getdestroyval(self, item):
        actions = []
        for eid, entity in enumerate(item['entities']):
            for sid, step in enumerate(item['steps']):
                o = 0
                c = 0
                d = 0
                if sid == 0:
                    prev_state = item['entity_step'][eid][sid]
                    continue
                else:
                    o += (prev_state[0] * item['entity_step'][eid][sid][0])
                    o += (prev_state[0] * item['entity_step'][eid][sid][1])
                    o += (prev_state[1] * item['entity_step'][eid][sid][0])
                    o += (prev_state[1] * item['entity_step'][eid][sid][1])
                    o += (prev_state[2] * item['entity_step'][eid][sid][2])
                    d += (prev_state[0] * item['entity_step'][eid][sid][2])
                    d += (prev_state[1] * item['entity_step'][eid][sid][2])
                    c += (prev_state[2] * item['entity_step'][eid][sid][1])
                    c += (prev_state[2] * item['entity_step'][eid][sid][0])
                    actions.append([1-d, d])
                    prev_state = item['entity_step'][eid][sid]
        return actions
    
    def getotherval(self, item):
        actions = []
        for eid, entity in enumerate(item['entities']):
            for sid, step in enumerate(item['steps']):
                o = 0
                c = 0
                d = 0
                if sid == 0:
                    prev_state = item['entity_step'][eid][sid]
                    continue
                else:
                    o += (prev_state[0] * item['entity_step'][eid][sid][0])
                    o += (prev_state[0] * item['entity_step'][eid][sid][1])
                    o += (prev_state[1] * item['entity_step'][eid][sid][0])
                    o += (prev_state[1] * item['entity_step'][eid][sid][1])
                    o += (prev_state[2] * item['entity_step'][eid][sid][2])
                    actions.append([1-o, o])
                    prev_state = item['entity_step'][eid][sid]
        return actions
    
    def getbeforeval(self, item):
        b1s = []
        b2s = []
        for step in range(len(item['steps'])):
            b1 = torch.zeros(len(item['steps']))
            b1[step] = 1
            for step1 in range(step+1, len(item['steps'])):
                b2 = torch.zeros(len(item['steps']))
                b2[step1] = 1
                b1s.append(b1)
                b2s.append(b2)
        return torch.stack(b1s), torch.stack(b2s)
    

In [6]:
ReaderObjectsIterator = ProparaReader("test_data.json")
next(iter(ReaderObjectsIterator))

{'action': (tensor([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
          [0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 0., 0., 0.

In [7]:
from regr.graph import Graph, Concept, Relation
from regr.graph.logicalConstrain import orL, andL, existsL, notL, atLeastL, atMostL, ifL

Graph.clear()
Concept.clear()
Relation.clear()

with Graph('global') as graph:
    procedure = Concept("procedure")
    step = Concept("step")
    (procedure_contain_step, ) = procedure.contains(step)
    word = Concept("word")
    (step_contains_word, ) = step.contains(word)
    entity = Concept("entity")
    entity_step = Concept("entity_step")
    (entity_of_step, step_of_entity) = entity_step.has_a(arg1=entity, arg2=step)
    entity_step_word = Concept("entity_step_word")
    (entity_of_step_word, step_of_entity_word, word_of_entity_step) = entity_step.has_a(arg1=entity, arg2=step, arg3=word)
    location_start = entity_step_word("start_location")
    location_end = entity_step_word("end_location")
    non_existence = entity_step("none_existence")
    unknown_loc = entity_step("unknown_location")
    known_loc = entity_step("known_location")
    before = Concept("before")
    (before_arg1, before_arg2) = before.has_a(arg1=step, arg2=step)
    action = Concept("action")
    (action_arg1, action_arg2) = action.has_a(arg1=entity_step, arg2=entity_step)
    create = action("create")
    destroy = action("create")
    other = action("create")

    # No entity_step




Log file for dataNode is in: /home/hfaghihi/Framework/DomiKnowS/examples/Propara/datanode.log


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:16: UserWarning: Please use OrderedDict rather than dict to prevent unpredictable order of arguments.For this instance, OrderedDict([('arg1', 'entity'), ('arg2', 'step')]) is used.
  app.launch_new_instance()
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:18: UserWarning: Please use OrderedDict rather than dict to prevent unpredictable order of arguments.For this instance, OrderedDict([('arg1', 'entity'), ('arg2', 'step'), ('arg3', 'word')]) is used.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:25: UserWarning: Please use OrderedDict rather than dict to prevent unpredictable order of arguments.For this instance, OrderedDict([('arg1', 'step'), ('arg2', 'step')]) is used.
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:27: UserWarning: Please use OrderedDict rather than dict to prevent unpredictable order of arguments.For this instance, OrderedDict([('arg1', 'entity_step'), ('arg2', 'e

In [8]:
from regr.sensor.pytorch.sensors import ReaderSensor, JointSensor
from regr.sensor.pytorch.relation_sensors import EdgeSensor

class JoinReaderSensor(ReaderSensor, JointSensor):
    pass
            
class JoinEdgeReaderSensor(JoinReaderSensor, EdgeSensor):
    def __init__(self, *pres, keyword=None, edges=None, forward=None, label=False, as_tensor=True, device='auto', relation=None, mode=None):
        JoinReaderSensor.__init__(self, *pres, edges=edges, forward=forward, label=label, as_tensor=as_tensor, device=device)
        EdgeSensor.__init__(self, *pres, relation=relation, mode=mode)

In [9]:
from regr.sensor.pytorch.sensors import ReaderSensor
from regr.program import LearningBasedProgram
from regr.program.model.pytorch import PoiModel
import torch
from torch import nn

def model_declaration():
    from graph import graph, procedure, word, step, entity, entity_step, entity_step_word, location_start, location_end, non_existence, unknown_loc, known_loc, action, create, destroy, other
    from graph import entity_of_step, entity_of_step_word, step_contains_word, step_of_entity, step_of_entity_word, word_of_entity_step, procedure_contain_step, action_arg1, action_arg2

    graph.detach()

    # --- City
    procedure['id'] = ReaderSensor(keyword='procedureID')
    step[procedure_contain_step.forward, 'text'] = JoinReaderSensor(procedure['id'], keyword='steps')
    # word[step_contains_word, 'raw'] = ReaderSensor(keyword='words')
    entity['raw'] = ReaderSensor(keyword='entities')
    entity_step[entity_of_step.forward, step_of_entity.forward, 'raw'] = JoinReaderSensor(entity['raw'], step['text'], step[procedure_contain_step.forward], keyword='entity_step')
    entity_step[non_existence] = ReaderSensor(entity_of_step.forward, step_of_entity.forward, 'raw', keyword='non_existence')
    entity_step[unknown_loc] = ReaderSensor(entity_of_step.forward, step_of_entity.forward, 'raw', keyword='known')
    entity_step[known_loc] = ReaderSensor(entity_of_step.forward, step_of_entity.forward, 'raw', keyword='unkown')
    entity_step[non_existence] = ReaderSensor(entity_of_step.forward, step_of_entity.forward, 'raw', keyword='non_existence')
    entity_step[unknown_loc] = ReaderSensor(entity_of_step.forward, step_of_entity.forward, 'raw', keyword='known')
    entity_step[known_loc] = ReaderSensor(entity_of_step.forward, step_of_entity.forward, 'raw', keyword='unkown')
    action[action_arg1.forward, action_arg2.forward] = JoinReaderSensor(entity_step[entity_of_step.forward], entity_step[step_of_entity.forward], entity_step['raw'], keyword='action')
    action[create] = ReaderSensor(action_arg1.forward, action_arg2.forward, keyword='create')
    action[destroy] = ReaderSensor(action_arg1.forward, action_arg2.forward, keyword='destroy')
    action[other] = ReaderSensor(action_arg1.forward, action_arg2.forward, keyword='other')
    action[create] = ReaderSensor(action_arg1.forward, action_arg2.forward, keyword='create')
    action[destroy] = ReaderSensor(action_arg1.forward, action_arg2.forward, keyword='destroy')
    action[other] = ReaderSensor(action_arg1.forward, action_arg2.forward, keyword='other')
    # entity_step_word[entity_of_step_word, step_of_entity_word, word_of_entity_step] = ReaderSensor(keyword='entity_step_word')
    # entity_step_word[location_start] = ReaderSensor(keyword='location_start')
    # entity_step_word[location_end] = ReaderSensor(keyword='location_end')
    program = LearningBasedProgram(graph, **{
        'Model': PoiModel,
#         'poi': (known_loc, unknown_loc, non_existence, other, destroy, create),
        'loss': None,
        'metric': None,
    })
    return program

In [10]:
def main():
    from graph import graph, procedure, word, step, entity, entity_step, entity_step_word, location_start, location_end, non_existence, unknown_loc, known_loc, action, create, destroy, other
    from graph import entity_of_step, entity_of_step_word, step_contains_word, step_of_entity, step_of_entity_word, word_of_entity_step, procedure_contain_step, action_arg1, action_arg2

    # set logger level to see training and testing logs
    import logging
    logging.basicConfig(level=logging.INFO)

    lbp = model_declaration()

    dataset = ProparaReader(file='test_data.json')  # Adding the info on the reader

#     lbp.test(dataset, device='auto')

    for datanode in lbp.populate(dataset):
        print('datanode:', datanode)
#         print('Spam:', datanode.getAttribute(Spam).softmax(-1))
#         print('Regular:', datanode.getAttribute(Regular).softmax(-1))
        datanode.inferILPConstrains(fun=lambda val: torch.tensor(val).softmax(dim=-1).detach().cpu().numpy().tolist(),
                                    epsilon=None)
        print('datanode:', datanode)
#         print('inference spam:', datanode.getAttribute(Spam, 'ILP'))
#         print('inference regular:', datanode.getAttribute(Regular, 'ILP'))

In [11]:
main()

/home/hfaghihi/Framework/DomiKnowS/examples/Propara/graph.py:16: UserWarning: Please use OrderedDict rather than dict to prevent unpredictable order of arguments.For this instance, OrderedDict([('arg1', 'entity'), ('arg2', 'step')]) is used.
  (entity_of_step, step_of_entity) = entity_step.has_a(arg1=entity, arg2=step)
/home/hfaghihi/Framework/DomiKnowS/examples/Propara/graph.py:18: UserWarning: Please use OrderedDict rather than dict to prevent unpredictable order of arguments.For this instance, OrderedDict([('arg1', 'entity'), ('arg2', 'step'), ('arg3', 'word')]) is used.
  (entity_of_step_word, step_of_entity_word, word_of_entity_step) = entity_step.has_a(arg1=entity, arg2=step, arg3=word)
/home/hfaghihi/Framework/DomiKnowS/examples/Propara/graph.py:25: UserWarning: Please use OrderedDict rather than dict to prevent unpredictable order of arguments.For this instance, OrderedDict([('arg1', 'step'), ('arg2', 'step')]) is used.
  (before_arg1, before_arg2) = before.has_a(arg1=step, arg

Error during updating data item with sensor functionalsensor-1
Error during updating data item with sensor readersensor-2


KeyError: 'entity_stepRelationAttrsCache'